# Industry energy demand

### Import necessary modules

In [1]:
# Check if we are running the notebook directly, if so move workspace to parent dir
import sys
import os
currentdir = os.path.abspath(os.getcwd())
if os.path.basename(currentdir) != 'DemandMappingZambia':  
  sys.path.insert(0, os.path.dirname(currentdir))
  os.chdir('..')
  print(f'Move to {os.getcwd()}')

Move to C:\Users\amillot\PycharmProjects\DemandMappingZambia


In [2]:
### Activate geospatial_env first

# Numeric
import numpy as np
import pandas as pd
import math

# System
import os
import shutil
from IPython.display import display, Markdown, HTML, FileLink, FileLinks

# Spatial
import geopandas as gpd
import json
import pyproj
from shapely.geometry import Point, Polygon, MultiPoint
from shapely.wkt import dumps, loads
from shapely.ops import nearest_points
from pyproj import CRS
import ogr, gdal, osr
#import fiona


# Mapping / Plotting
from functools import reduce
#import datapane as dp 
#!datapane login --token="9bde41bfbc4ad14119e32086f9f06d2e5db1d5b8"
import folium
from folium.features import GeoJsonTooltip
from folium.plugins import BeautifyIcon
from folium.plugins import HeatMap
import branca.colormap as cm
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib inline

%matplotlib inline

In [3]:
#import geopandas as gpd   # Note that you require geopandas version >= 0.7 that incluse clip see here for installation (https://gis.stackexchange.com/questions/360127/geopandas-0-6-1-installed-instead-of-0-7-0-in-conda-windows-10#)
import os
import fiona
import ipywidgets as widgets
from IPython.display import display
from rasterstats import zonal_stats
import rasterio
from geojson import Feature, Point, FeatureCollection
import rasterio.fill
from shapely.geometry import shape, mapping
import json
#from earthpy import clip    clip has been deprecated to geopandas
#import earthpy.spatial as es
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
import gdal
import datetime
import warnings
import pandas as pd
import scipy.spatial
warnings.filterwarnings('ignore')

#import contextily as ctx
import h3 as h3
from shapely.ops import unary_union
from shapely.geometry.polygon import Polygon

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

pd.options.display.float_format = '{:,.4f}'.format

In [4]:
from utils import processing_raster, finalizing_rasters, spatialjoinvectors

### Define directories and dataset names

In [5]:
### Define directories and dataset names
ROOT_DIR = os.path.abspath(os.curdir)
in_path = ROOT_DIR
out_path = ROOT_DIR + "/Outputs"

## admininstrative boundary
admin_path = in_path + "\\"+ 'admin'
admin_name = "Copperbelt.gpkg"
adm_col_name = "ADM1_NAME"          ## Provide the name of the column you want to use to clip the hexagons e.g., "NAME" or "ADM1_NAME"

## mines layer
mines_path = in_path + "/Industry/Data/mines"
mines_name = 'mines_zambia.gpkg'

In [6]:
## Coordinate and projection systems
crs_WGS84 = CRS("EPSG:4326")    # Originan WGS84 coordinate system
crs_proj = CRS("EPSG:32736")    # Projection system for the selected country -- see http://epsg.io/ for more info

### Import layers to be used

In [7]:
hexagons = gpd.read_file(out_path + "\\" + "h3_grid_at_hex_7.shp")

In [8]:
grid = hexagons

In [9]:
## mines
mines = gpd.read_file(mines_path + "/" + mines_name)

In [10]:
## admininstrative boundary
admin_gdf = gpd.read_file(admin_path + "/" + admin_name)

# Part 1. Extract GIS-based attributes

## Extract information from vector layers

Extract sum production of mines in each cluster (hex)

In [11]:
## Run the extraction
#grid.drop(['Commodity Production - tonne (tonnes)'], axis=1, inplace=True) ##uncomment if you want to rerun
# columnNameMines = "Commodity Production - tonne (tonnes)"
columnNameMines = 'Ore processed (tonnes)'
grid, mines = spatialjoinvectors("Mines", columnNameMines, admin_gdf, crs_WGS84, grid, "sum")
grid[columnNameMines] = grid[columnNameMines].fillna(0)
grid.head(4)

,hexagons,lon,lat,index_righ,ADM1_NAME,id,geometry,Ore processed (tonnes)
0,873469a43ffffff,28.5112,-13.0582,0.0000,COPPERBELT,1,"POLYGON ((28.52398 -13.06274, 28.51388 -13.073...",0.0000
1,875536acdffffff,27.3135,-13.4529,0.0000,COPPERBELT,2,"POLYGON ((27.32628 -13.45739, 27.31616 -13.467...",0.0000
2,87553411bffffff,27.9715,-12.7327,0.0000,COPPERBELT,3,"POLYGON ((27.98428 -12.73706, 27.97422 -12.747...","1,465,667.0000"
3,875534d8effffff,28.3352,-13.0799,0.0000,COPPERBELT,4,"POLYGON ((28.34798 -13.08439, 28.33788 -13.094...",0.0000


In [12]:
grid[columnNameMines].sum()

24359691.0

# Part 2. Compute demand

In [13]:
# Assess total energy consumption and total production
elec_nonFerrousMetals = 22897 #PJ UN stats
elec_nonFerrousMetals = elec_nonFerrousMetals / (3.6e-6) #conversion in MWh
total_production = mines[columnNameMines].str.replace(',', '').astype(float).sum()
# total_production = sum(grid[columnNameMines]) # ton
energycons_perton = elec_nonFerrousMetals/total_production # MWh/t
print("total production:", f"{total_production/10**3:,.0f}", "kt")
print("total industry electricity consumption:",f"{elec_nonFerrousMetals/10**6:,.0f}", "TWh")
print("energy per tonne of ore:", f"{energycons_perton:,.0f}", "MWh/t")

total production: 128,215 kt
total industry electricity consumption: 6,360 TWh
energy per tonne of ore: 50 MWh/t


In [14]:
#Allocate to each hexagon the industry energy consumption
grid["IndEnergy"]=grid[columnNameMines]*energycons_perton
grid.head(3)
total_industryenergy = grid["IndEnergy"].sum()
print("Industry electricity consumption:",f"{total_industryenergy/10**6:,.0f}", "TWh")

Industry electricity consumption: 1,208 TWh


In [15]:
grid.to_file(out_path + "\\" + 'ind_energy_map.shp', index=False)
grid.head(3)

,hexagons,lon,lat,index_righ,ADM1_NAME,id,geometry,Ore processed (tonnes),IndEnergy
0,873469a43ffffff,28.5112,-13.0582,0.0000,COPPERBELT,1,"POLYGON ((28.52398 -13.06274, 28.51388 -13.073...",0.0000,0.0000
1,875536acdffffff,27.3135,-13.4529,0.0000,COPPERBELT,2,"POLYGON ((27.32628 -13.45739, 27.31616 -13.467...",0.0000,0.0000
2,87553411bffffff,27.9715,-12.7327,0.0000,COPPERBELT,3,"POLYGON ((27.98428 -12.73706, 27.97422 -12.747...","1,465,667.0000","72,706,496.3342"
